# Wave equation

In this tutorial we present how to solve a wave equation in mixed form with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the particle velocity $\tau$ and the acustic pressure $v$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the square of the characteristic velocity, we want to solve the following problem: find $({\tau}, v)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} \partial_t {\tau} + \nabla v = {0}\\
\nabla \cdot {\tau} + \partial_t v = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ v = 0 \text{ on } \partial_{top} \Omega \qquad v = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot \tau = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we will use a Raviart-Thomas approximation for ${\tau}$ we are restricted to simplices. In this example we consider a bi-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [2]:
N = 10
sd = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([sd])

We consider an Implicit Euler scheme for the time advancing, let's set the relative data

In [3]:
T = 1
num_steps = 10
delta_t = T / num_steps

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [4]:
keyword = "acustic"
bc_val, bc_ess = [], []
for sd, data in mdg.subdomains(return_data=True):
    # square of the characteristic velocity
    vel = pp.SecondOrderTensor(np.ones(sd.num_cells))
    
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    b_faces = sd.tags["domain_boundary_faces"].nonzero()[0] 
    
    b_face_centers = sd.face_centers[:, b_faces]
    out_flow = b_face_centers[1, :] == 1
    in_flow = b_face_centers[1, :] == 0
    no_flow = np.logical_or.reduce((b_face_centers[0, :] == 1, b_face_centers[0, :] == 0))

    faces, _, sign = sps.find(sd.cell_faces)
    sign = sign[np.unique(faces, return_index=True)[1]]
        
    ess_faces = np.zeros(sd.num_faces, dtype=bool)
    ess_faces[b_faces[no_flow]] = True
    ess_cells = np.zeros(sd.num_cells, dtype=bool)    

    bc_faces = np.zeros(sd.num_faces)
    bc_faces[b_faces[in_flow]] = sign[b_faces[in_flow]] * np.ones(b_faces[in_flow].size)
    
    bc_val.append(np.hstack((bc_faces, np.zeros(sd.num_cells))))
    bc_ess.append(np.hstack((ess_faces, ess_cells)))
    
    parameters = {
        "second_order_tensor": vel,
    }
    data[pp.PARAMETERS] = {keyword: parameters}
    data[pp.DISCRETIZATION_MATRICES] = {keyword: {}}
    
bc_val = np.hstack(bc_val)
bc_ess = np.hstack(bc_ess)

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system associated with the equation is given as at time step $n$
$$
\left(
\begin{array}{cc} 
M_{face} & -B^\top\\
B & M_{cell}
\end{array}
\right)
\left(
\begin{array}{c} 
\tau^{n+1}\\ 
v^{n+1}
\end{array}
\right)
=\left(
\begin{array}{c} 
v_{\partial}\\ 
0
\end{array}
\right)+
\left(
\begin{array}{cc} 
M_{face} & 0\\
0 & M_{cell}
\end{array}
\right)
\left(
\begin{array}{c} 
\tau^n\\ 
v^n
\end{array}
\right)
$$<br>
where $v_{\partial}$ is the vector associated to the pressure boundary contions. To construct the problem, we rely on the `scipy.sparse` function `bmat`. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [6]:
# construct the local matrices
face_mass = pg.face_mass(mdg, keyword=keyword)/delta_t
cell_mass = pg.cell_mass(mdg, keyword=keyword)/delta_t
div = pg.div(mdg)

# assemble the problem matrices
mat_rhs = sps.bmat([[face_mass,    -div.T], 
                    [      div, cell_mass]], format="csc")

mat_lhs = sps.bmat([[face_mass,      None], 
                    [     None, cell_mass]], format="csc")

# get the degrees of freedom for each variable
dof_v, dof_tau = div.shape
dofs = np.cumsum(dof_tau)

# assemble the right-hand side
rhs = bc_val

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (flux boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom. Once the problem is solved, we extract the two solutions $q$ and $p$.

In [27]:
# solve the problem
x = np.zeros(mat.shape[0])
for n in np.arange(N):
    ls = pg.LinearSystem(mat_rhs, rhs + mat_lhs * x)
    ls.flag_ess_bc(np.hstack(bc_ess), np.zeros(bc_ess.size))
    x = ls.solve()

    # extract the variables
    tau, v = np.split(x, dofs)

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [8]:
# post process velocity
face_proj = pg.proj_faces_to_cells(mdg)
cell_q = (face_proj * q).reshape((3, -1), order="F")

for _, data in mdg.subdomains(return_data=True):
    data[pp.STATE] = {"cell_q": cell_q, "p": p}

save = pp.Exporter(mdg, "sol")
save.write_vtu(["cell_q", "p"])

A representation of the computed solution is given below, where the cells are colored with $p$ and the arrows are the $q$. <br>
![](fig/darcy.png)